In [1]:
import dreamer as dm
import numpy as np
import sys
import os
import torch
import matplotlib.pyplot as plt

import gymnasium as gym
from importlib import reload
import time
from tqdm import tqdm

sys.path.append(os.path.abspath('../hockey_env/hockey'))
import hockey_env as h_env


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
#env = h_env.HockeyEnv()
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_SHOOTING)
_ = env.render()

dreamer = dm.DreamerV3(
	env=env,
    render=True,
    obs_dim=env.observation_space.shape[0],
    action_dim=env.action_space.shape[0] // 2,
    latent_dim=32,
    latent_categories_size=32,
    model_dim=256,
    num_blocks=8,
    imagination_horizon=15,
    capacity=10000,
    replay_ratio=32,
    imag_horizon=15,
    bins=15,
    min_reward=-5,
    max_reward=5,
    device=device)  

replay_ratio = 32

number_of_training_steps = 20000
batch_size = 16
seq_len = 64
number_of_trajectories = 25
max_steps = 100

env_step_per_training_step = batch_size * seq_len // replay_ratio 
start_samples = 1200

world_losses = []
critic_losses = []
actor_losses = []
rewards = []

dreamer.generate_samples(start_samples)

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


np.float64(-0.2993589396934295)

In [ ]:
for step in range(number_of_training_steps):
    rew = dreamer.generate_samples(env_step_per_training_step)

    world_loss, critic_loss, actor_loss = dreamer.train(batch_size, seq_len)
    world_losses.append(world_loss)
    critic_losses.append(critic_loss)
    actor_losses.append(actor_loss)
    rewards.append(rew)
    print(f"Step {step} Reward:{rew: .4f} Losses: {world_loss[0].item():.2f} {world_loss[1].item():.2f} {world_loss[2].item():.2f}    {critic_loss:.2f}   {actor_loss:.2f}")

Step 0 Reward:-0.2679 Losses: 6.89 0.00 0.00    2.78   -0.47
Step 1 Reward:-0.2543 Losses: 7.18 0.00 0.00    2.61   0.23
Step 2 Reward:-0.3237 Losses: 6.78 0.00 0.00    2.41   0.95
Step 3 Reward:-0.3500 Losses: 5.70 0.00 0.00    2.16   1.53
Step 4 Reward:-0.2845 Losses: 5.68 0.01 0.00    1.85   1.95
Step 5 Reward:-0.4127 Losses: 5.19 0.01 0.00    1.45   1.43
Step 6 Reward:-0.2285 Losses: 3.32 0.02 0.00    0.98   0.60
Step 7 Reward:-0.0601 Losses: 3.71 0.05 0.00    0.61   0.04
Step 8 Reward:-0.2128 Losses: 5.28 0.07 0.01    0.42   -0.19
Step 9 Reward:-0.3266 Losses: 4.23 0.08 0.01    0.34   -0.29
Step 10 Reward:-0.2305 Losses: 3.21 0.07 0.01    0.32   -0.42
Step 11 Reward:-0.1065 Losses: 4.14 0.05 0.00    0.32   -0.63
Step 12 Reward:-0.3456 Losses: 5.53 0.03 0.00    0.36   -0.80
Step 13 Reward:-0.4462 Losses: 4.19 0.02 0.00    0.42   -1.23
Step 14 Reward:-0.3634 Losses: 4.10 0.02 0.00    0.52   -1.78
Step 15 Reward:-0.5411 Losses: 5.75 0.01 0.00    0.68   -2.58
Step 16 Reward:-0.5358 Lo

In [ ]:
# Plot the rewards
#plt.plot(rewards)
#plt.show()

: 

# Check the agent

In [5]:
obs, info = env.reset()
obs_agent2 = env.obs_agent_two()
_ = env.render()


for i in range(100):
	env_reset = True
	for _ in range(6000):
		if env_reset:
			obs_0, info = env.reset()
			h_0 = dreamer.world_model.get_default_hidden()
			env_reset = False
		env.render(mode="human")

		obs_0 = torch.tensor(obs_0, dtype=torch.float32, device=device)
		z_0 = dreamer.world_model.get_latent(h_0, obs_0)
		a_enemy = np.random.uniform(-1,1,4)
		a_0 = dreamer.actor.get_action(h_0, z_0)
		#a_0 = torch.tensor([1, 0.0, 0.0, 0.0], dtype=torch.float32, device=device)
		a_0_numpy = a_0.cpu().detach().numpy()
		obs_1, r, done, t, info = env.step(np.hstack([a_0_numpy,a_enemy]))    
		h_0 = dreamer.world_model.get_recurrent_hidden(h_0, z_0, a_0)
		if done or t:
			print(done, t)
			break
		

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
C:\Users\pmaty\AppData\Local\Temp\ipykernel_22232\2294970489.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs_0 = torch.tensor(obs_0, dtype=torch.float32, device=device)


True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False
True False


KeyboardInterrupt: 

In [10]:
_

250